In [2]:
from zmax_datasets.datasets.usleep import USleepDataset
from pathlib import Path
import numpy as np

DATASETS_DIR = Path("/project/4180000.46/zmax_datasets/accelerometer_usleep")
DATASET_NAME = "donders_2018"

dataset = USleepDataset(data_dir=DATASETS_DIR / DATASET_NAME)
print(dataset.n_recordings)
print(dataset.recording_ids)


6
['P8-night3', 'P11-night3', 'P13-night3', 'P18-night3', 'P20-night3', 'P21-night2']


In [3]:
# Get a sample recording
sample_recording = dataset.get_recording("P8-night3")
print("Available data types:")
print(sample_recording.data_types)


Available data types:
{'ACCELEROMETER_X_dynamic': DataType(channel='ACCELEROMETER_X_dynamic', sampling_rate=64.0), 'ACCELEROMETER_Y_dynamic': DataType(channel='ACCELEROMETER_Y_dynamic', sampling_rate=64.0), 'ACCELEROMETER_Z_dynamic': DataType(channel='ACCELEROMETER_Z_dynamic', sampling_rate=64.0), 'ACCELEROMETER_magnitude': DataType(channel='ACCELEROMETER_magnitude', sampling_rate=64.0), 'ACCELEROMETER_magnitude_derivative': DataType(channel='ACCELEROMETER_magnitude_derivative', sampling_rate=64.0), 'NOISE': DataType(channel='NOISE', sampling_rate=64.0)}


In [4]:
# Load noise signal
noise_signal = sample_recording.read_data_type("NOISE")

# Create time axis in seconds for the signal
duration_seconds = len(noise_signal.array.squeeze()) / noise_signal.sample_rate
time = np.linspace(0, duration_seconds, len(noise_signal.array.squeeze()))
print(f"\nSignal duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")
print(f"Sampling rate: {noise_signal.sample_rate} Hz")


2025-09-05 13:33:09.454 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: NOISE



Signal duration: 29720.0 seconds (495.3 minutes)
Sampling rate: 64.0 Hz


In [5]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create widgets for controlling the window
start_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=duration_seconds - 60,  # Leave room for the window
    step=10,
    description='Start Time (s):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

window_size = widgets.Dropdown(
    options=[('10 seconds', 10), ('30 seconds', 30), ('1 minute', 60), ('5 minutes', 300)],
    value=60,
    description='Window Size:',
    style={'description_width': 'initial'}
)

# Create initial figure
fig = go.FigureWidget(make_subplots(
    rows=1, cols=1,
    vertical_spacing=0.05,
    subplot_titles=("Noise Signal",)
))

# Initialize with empty trace
fig.add_trace(go.Scatter(name="Noise", line=dict(color='blue')), row=1, col=1)

# Update layout
fig.update_layout(
    height=600,
    title=f"Noise Signal Analysis - Recording {str(sample_recording)}",
    showlegend=True,
    template="plotly_white"
)

# Update axes labels
fig.update_yaxes(title_text="Noise Level", row=1, col=1)
fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)

def update_plot(start_time, window_duration):
    # Calculate indices for the window
    start_idx = int(start_time * noise_signal.sample_rate)
    end_idx = int((start_time + window_duration) * noise_signal.sample_rate)
    
    # Create time array for the window
    time_window = np.linspace(start_time, start_time + window_duration, end_idx - start_idx)
    
    # Get signal segment
    noise_segment = noise_signal.array.squeeze()[start_idx:end_idx]
    
    # Update trace with new data
    with fig.batch_update():
        fig.data[0].x = time_window
        fig.data[0].y = noise_segment

# Create the interactive plot
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_plot(start_slider.value, window_size.value)

# Link the widgets to the update function
start_slider.observe(on_change)
window_size.observe(on_change)

# Display widgets and initial plot
display(widgets.HBox([start_slider, window_size]))
display(fig)

# Initialize the plot
update_plot(start_slider.value, window_size.value)


FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'name': 'Noise',
              'type': 'scatter',
              'uid': 'e2d73a1c-aa25-412b-9fb9-ad12e4867913',
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Noise Signal',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'height': 600,
               'showlegend': True,
               'template': '...',
               'title': {'text': 'Noise Signal Analysis - Recording P8-night3'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Time (seconds)'}},
               'yaxis': {'anc